In [1]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
# text in Western (Windows 1252)

import pickle
import numpy as np
# import StringIO
import math
from keras import optimizers, metrics
from keras.models import Model
from keras.layers import Dense, Dropout, Input
from keras.layers.merge import concatenate
from keras import regularizers
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.constraints import maxnorm
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.models import load_model
# from keras import backend as Input
np.random.seed(7)

Using Theano backend.


In [2]:
# %run ../../../prepare_data.py

import sys
sys.path.insert(0, '../../../')
from prepare_data import *

In [3]:
# %run ../../../prepare_data.py
# X_train, X_other_features_train, y_train, X_test, X_other_features_test, y_test, X_validate, X_other_features_validate, y_validate = generate_full_matrix_inputs('../../internal_representations/inputs/content_shuffle_vector.h5', '../../internal_representations/inputs/shuffle_vector')
# save_inputs('../../internal_representations/inputs/shuffled_letters_train.h5', X_train, y_train, other_features = X_other_features_train)
# save_inputs('../../internal_representations/inputs/shuffled_letters_test.h5', X_test, y_test, other_features = X_other_features_test)
# save_inputs('../../internal_representations/inputs/shuffled_letters_validate.h5', X_validate, y_validate,  other_features = X_other_features_validate)
# X_train, X_other_features_train, y_train = load_inputs('../../internal_representations/inputs/shuffled_letters_train.h5', other_features=True)
# X_test, X_other_features_test, y_test = load_inputs('../../internal_representations/inputs/shuffled_letters_test.h5', other_features=True)
# X_validate, X_other_features_validate, y_validate = load_inputs('../../internal_representations/inputs/shuffled_letters_validate.h5', other_features=True)

data = Data('l')
data.generate_data('letters_word_accetuation_train',
                   'letters_word_accetuation_test',
                   'letters_word_accetuation_validate')

READING CONTENT...
CONTENT READ SUCCESSFULLY
CREATING DICTIONARY...
DICTIONARY CREATION SUCCESSFUL!
GENERATING X AND y...
CREATING OTHER FEATURES...
OTHER FEATURES CREATED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
CREATING OTHER FEATURES...
OTHER FEATURES CREATED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
CREATING OTHER FEATURES...
OTHER FEATURES CREATED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
GENERATION SUCCESSFUL!


In [4]:
gen2 = data.generator('train', 16)
# test1 = next(gen1)
test2 = next(gen2)

In [7]:
pos = 0
print(len(feature_dictionary))
print(np.array(test2[0][0]).shape)
print(np.array(test2[0][1]).shape)
# for el in test2[0][0]:
# #     print(el)
#     print (data.decode_x(el, dictionary))
#     print(data.decode_x_other_features(feature_dictionary, [el]))

# for el in test2[1]:
#     print(el)

10
(16, 23, 36)
(16, 140)


In [21]:
# print (X_train.shape)
# print (X_test.shape)
# print (X_validate.shape)
pos = 7
# print (decode_input(X_train[pos], dictionary))
# print (decode_X_features(feature_dictionary, [X_other_features_train[pos]]))
# print(decode_position(y_train[pos]))
# print('------------------------------------')
print (data.x_train.shape)
print (data.x_test.shape)
print (data.x_validate.shape)
# pos = 2
print (data.decode_x(data.x_train[pos], dictionary))
print (data.decode_x_other_features(feature_dictionary, [data.x_other_features_train[pos]]))
print(data.decode_y(data.y_train[pos]))

(430151, 23, 36)
(52058, 23, 36)
(54222, 23, 36)
amejnavejob
Ncndi-
Ncndi-
[3]


In [28]:
num_examples = len(data.x_train) # training set size
nn_output_dim = 10
nn_hdim = 516
batch_size = 16
actual_epoch = 40
num_fake_epoch = 20




conv_input_shape=(23, 36)
othr_input = (140, )

conv_input = Input(shape=conv_input_shape, name='conv_input')
x_conv = Conv1D(115, (3), padding='same', activation='relu')(conv_input)
x_conv = Conv1D(46, (3), padding='same', activation='relu')(x_conv)
x_conv = MaxPooling1D(pool_size=2)(x_conv)
x_conv = Flatten()(x_conv)
# x_conv = Dense(516, activation='relu', kernel_constraint=maxnorm(3))(x_conv)

othr_input = Input(shape=othr_input, name='othr_input')
# x_othr = Dense(256, input_dim=167, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(512, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(256, activation='relu')(othr_input)

x = concatenate([x_conv, othr_input])
# x = Dense(1024, input_dim=(516 + 256), activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.4)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.4)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(nn_output_dim, activation='sigmoid')(x)




model = Model(inputs=[conv_input, othr_input], outputs=x)
opt = optimizers.Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[actual_accuracy,])
# model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [29]:
history = model.fit_generator(data.generator('train', batch_size), 
                              data.x_train.shape[0]/(batch_size * num_fake_epoch), 
                              epochs=actual_epoch*num_fake_epoch, 
                              validation_data=data.generator('test', batch_size), 
                              validation_steps=data.x_test.shape[0]/(batch_size * num_fake_epoch))

Epoch 1/800
1345/1344 [==============================] - 27s - loss: 0.2171 - actual_accuracy: 0.3783 - val_loss: 0.1567 - val_actual_accuracy: 0.6089
Epoch 2/800
1345/1344 [==============================] - 28s - loss: 0.1599 - actual_accuracy: 0.5465 - val_loss: 0.1361 - val_actual_accuracy: 0.6400
Epoch 3/800
1345/1344 [==============================] - 29s - loss: 0.1441 - actual_accuracy: 0.5967 - val_loss: 0.1242 - val_actual_accuracy: 0.6603
Epoch 4/800
1345/1344 [==============================] - 27s - loss: 0.1325 - actual_accuracy: 0.6431 - val_loss: 0.1112 - val_actual_accuracy: 0.7147
Epoch 5/800
1345/1344 [==============================] - 27s - loss: 0.1250 - actual_accuracy: 0.6711 - val_loss: 0.1104 - val_actual_accuracy: 0.7435
Epoch 6/800
1345/1344 [==============================] - 28s - loss: 0.1186 - actual_accuracy: 0.6953 - val_loss: 0.0984 - val_actual_accuracy: 0.7692
Epoch 7/800
1345/1344 [==============================] - 29s - loss: 0.1130 - actual_accuracy:

1345/1344 [==============================] - 28s - loss: 0.0383 - actual_accuracy: 0.9103 - val_loss: 0.0762 - val_actual_accuracy: 0.8773
Epoch 55/800
1345/1344 [==============================] - 32s - loss: 0.0366 - actual_accuracy: 0.9146 - val_loss: 0.0718 - val_actual_accuracy: 0.8880
Epoch 56/800
1345/1344 [==============================] - 35s - loss: 0.0362 - actual_accuracy: 0.9158 - val_loss: 0.0649 - val_actual_accuracy: 0.8911
Epoch 57/800
1345/1344 [==============================] - 36s - loss: 0.0339 - actual_accuracy: 0.9197 - val_loss: 0.0814 - val_actual_accuracy: 0.8800
Epoch 58/800
1345/1344 [==============================] - 36s - loss: 0.0348 - actual_accuracy: 0.9199 - val_loss: 0.0722 - val_actual_accuracy: 0.8897
Epoch 59/800
1345/1344 [==============================] - 35s - loss: 0.0336 - actual_accuracy: 0.9204 - val_loss: 0.0743 - val_actual_accuracy: 0.8823
Epoch 60/800
1345/1344 [==============================] - 37s - loss: 0.0337 - actual_accuracy: 0.923

1345/1344 [==============================] - 35s - loss: 0.0193 - actual_accuracy: 0.9558 - val_loss: 0.1046 - val_actual_accuracy: 0.8781
Epoch 108/800
1345/1344 [==============================] - 37s - loss: 0.0207 - actual_accuracy: 0.9529 - val_loss: 0.0952 - val_actual_accuracy: 0.8919
Epoch 109/800
1345/1344 [==============================] - 35s - loss: 0.0204 - actual_accuracy: 0.9526 - val_loss: 0.0782 - val_actual_accuracy: 0.9110
Epoch 110/800
1345/1344 [==============================] - 37s - loss: 0.0207 - actual_accuracy: 0.9526 - val_loss: 0.0978 - val_actual_accuracy: 0.8911
Epoch 111/800
1345/1344 [==============================] - 34s - loss: 0.0196 - actual_accuracy: 0.9565 - val_loss: 0.1035 - val_actual_accuracy: 0.8831
Epoch 112/800
1345/1344 [==============================] - 34s - loss: 0.0198 - actual_accuracy: 0.9549 - val_loss: 0.0879 - val_actual_accuracy: 0.8984
Epoch 113/800
1345/1344 [==============================] - 40s - loss: 0.0194 - actual_accuracy:

1345/1344 [==============================] - 40s - loss: 0.0150 - actual_accuracy: 0.9669 - val_loss: 0.0965 - val_actual_accuracy: 0.9022
Epoch 161/800
1345/1344 [==============================] - 37s - loss: 0.0153 - actual_accuracy: 0.9649 - val_loss: 0.0902 - val_actual_accuracy: 0.9068
Epoch 162/800
1345/1344 [==============================] - 39s - loss: 0.0145 - actual_accuracy: 0.9689 - val_loss: 0.0952 - val_actual_accuracy: 0.9030
Epoch 163/800
1345/1344 [==============================] - 39s - loss: 0.0149 - actual_accuracy: 0.9671 - val_loss: 0.0958 - val_actual_accuracy: 0.9041
Epoch 164/800
1345/1344 [==============================] - 39s - loss: 0.0144 - actual_accuracy: 0.9688 - val_loss: 0.1037 - val_actual_accuracy: 0.8903
Epoch 165/800
1345/1344 [==============================] - 36s - loss: 0.0146 - actual_accuracy: 0.9657 - val_loss: 0.1001 - val_actual_accuracy: 0.8846
Epoch 166/800
1345/1344 [==============================] - 42s - loss: 0.0135 - actual_accuracy:

1345/1344 [==============================] - 39s - loss: 0.0111 - actual_accuracy: 0.9748 - val_loss: 0.1012 - val_actual_accuracy: 0.8970
Epoch 214/800
1345/1344 [==============================] - 39s - loss: 0.0124 - actual_accuracy: 0.9724 - val_loss: 0.1072 - val_actual_accuracy: 0.9007
Epoch 215/800
1345/1344 [==============================] - 38s - loss: 0.0117 - actual_accuracy: 0.9742 - val_loss: 0.1060 - val_actual_accuracy: 0.9007
Epoch 216/800
1345/1344 [==============================] - 37s - loss: 0.0107 - actual_accuracy: 0.9756 - val_loss: 0.1088 - val_actual_accuracy: 0.9095
Epoch 217/800
1345/1344 [==============================] - 35s - loss: 0.0111 - actual_accuracy: 0.9757 - val_loss: 0.1101 - val_actual_accuracy: 0.8850
Epoch 218/800
1345/1344 [==============================] - 36s - loss: 0.0113 - actual_accuracy: 0.9745 - val_loss: 0.0959 - val_actual_accuracy: 0.9022
Epoch 219/800
1345/1344 [==============================] - 36s - loss: 0.0111 - actual_accuracy:

...

1345/1344 [==============================] - 40s - loss: 0.0063 - actual_accuracy: 0.9860 - val_loss: 0.1257 - val_actual_accuracy: 0.9007
Epoch 585/800
1345/1344 [==============================] - 34s - loss: 0.0061 - actual_accuracy: 0.9863 - val_loss: 0.1278 - val_actual_accuracy: 0.8980
Epoch 586/800
1345/1344 [==============================] - 38s - loss: 0.0061 - actual_accuracy: 0.9877 - val_loss: 0.1107 - val_actual_accuracy: 0.9057
Epoch 587/800
1345/1344 [==============================] - 36s - loss: 0.0060 - actual_accuracy: 0.9868 - val_loss: 0.0995 - val_actual_accuracy: 0.9103
Epoch 588/800
1345/1344 [==============================] - 37s - loss: 0.0055 - actual_accuracy: 0.9878 - val_loss: 0.1047 - val_actual_accuracy: 0.9103
Epoch 589/800
1345/1344 [==============================] - 39s - loss: 0.0063 - actual_accuracy: 0.9868 - val_loss: 0.1177 - val_actual_accuracy: 0.9049
Epoch 590/800
1345/1344 [==============================] - 35s - loss: 0.0060 - actual_accuracy:

1345/1344 [==============================] - 37s - loss: 0.0060 - actual_accuracy: 0.9884 - val_loss: 0.1314 - val_actual_accuracy: 0.8926
Epoch 638/800
1345/1344 [==============================] - 37s - loss: 0.0060 - actual_accuracy: 0.9871 - val_loss: 0.1136 - val_actual_accuracy: 0.9061
Epoch 639/800
1345/1344 [==============================] - 35s - loss: 0.0056 - actual_accuracy: 0.9870 - val_loss: 0.1316 - val_actual_accuracy: 0.8939
Epoch 640/800
1345/1344 [==============================] - 38s - loss: 0.0066 - actual_accuracy: 0.9859 - val_loss: 0.1135 - val_actual_accuracy: 0.8995
Epoch 641/800
1345/1344 [==============================] - 34s - loss: 0.0060 - actual_accuracy: 0.9873 - val_loss: 0.1170 - val_actual_accuracy: 0.9072
Epoch 642/800
1345/1344 [==============================] - 35s - loss: 0.0050 - actual_accuracy: 0.9896 - val_loss: 0.1060 - val_actual_accuracy: 0.9126
Epoch 643/800
1345/1344 [==============================] - 38s - loss: 0.0067 - actual_accuracy:

1345/1344 [==============================] - 38s - loss: 0.0051 - actual_accuracy: 0.9886 - val_loss: 0.1366 - val_actual_accuracy: 0.8988
Epoch 691/800
1345/1344 [==============================] - 36s - loss: 0.0052 - actual_accuracy: 0.9891 - val_loss: 0.1262 - val_actual_accuracy: 0.8999
Epoch 692/800
1345/1344 [==============================] - 37s - loss: 0.0054 - actual_accuracy: 0.9888 - val_loss: 0.1340 - val_actual_accuracy: 0.9038
Epoch 693/800
1345/1344 [==============================] - 37s - loss: 0.0058 - actual_accuracy: 0.9883 - val_loss: 0.1150 - val_actual_accuracy: 0.9041
Epoch 694/800
1345/1344 [==============================] - 36s - loss: 0.0061 - actual_accuracy: 0.9877 - val_loss: 0.1309 - val_actual_accuracy: 0.9038
Epoch 695/800
1345/1344 [==============================] - 37s - loss: 0.0058 - actual_accuracy: 0.9882 - val_loss: 0.1287 - val_actual_accuracy: 0.9007
Epoch 696/800
1345/1344 [==============================] - 41s - loss: 0.0053 - actual_accuracy:

1345/1344 [==============================] - 33s - loss: 0.0050 - actual_accuracy: 0.9892 - val_loss: 0.0985 - val_actual_accuracy: 0.9091
Epoch 744/800
1345/1344 [==============================] - 37s - loss: 0.0060 - actual_accuracy: 0.9878 - val_loss: 0.1259 - val_actual_accuracy: 0.9007
Epoch 745/800
1345/1344 [==============================] - 36s - loss: 0.0055 - actual_accuracy: 0.9878 - val_loss: 0.1297 - val_actual_accuracy: 0.8953
Epoch 746/800
1345/1344 [==============================] - 36s - loss: 0.0052 - actual_accuracy: 0.9887 - val_loss: 0.1172 - val_actual_accuracy: 0.9003
Epoch 747/800
1345/1344 [==============================] - 36s - loss: 0.0056 - actual_accuracy: 0.9882 - val_loss: 0.1160 - val_actual_accuracy: 0.9103
Epoch 748/800
1345/1344 [==============================] - 38s - loss: 0.0059 - actual_accuracy: 0.9871 - val_loss: 0.1211 - val_actual_accuracy: 0.8965
Epoch 749/800
1345/1344 [==============================] - 37s - loss: 0.0055 - actual_accuracy:

1345/1344 [==============================] - 37s - loss: 0.0053 - actual_accuracy: 0.9888 - val_loss: 0.1194 - val_actual_accuracy: 0.9095
Epoch 797/800
1345/1344 [==============================] - 37s - loss: 0.0055 - actual_accuracy: 0.9883 - val_loss: 0.1432 - val_actual_accuracy: 0.9072
Epoch 798/800
1345/1344 [==============================] - 37s - loss: 0.0054 - actual_accuracy: 0.9888 - val_loss: 0.1141 - val_actual_accuracy: 0.9103
Epoch 799/800
1345/1344 [==============================] - 37s - loss: 0.0057 - actual_accuracy: 0.9885 - val_loss: 0.1068 - val_actual_accuracy: 0.9072
Epoch 800/800
1345/1344 [==============================] - 38s - loss: 0.0054 - actual_accuracy: 0.9887 - val_loss: 0.1103 - val_actual_accuracy: 0.9107


In [27]:
name = '40_epoch'
model.save(name + '.h5')
output = open(name + '_history.pkl', 'wb')
pickle.dump(history.history, output)
output.close()

In [6]:
content = data._read_content('../../../data/SlovarIJS_BESEDE_utf8.lex')
dictionary, max_word, max_num_vowels, vowels, accented_vowels = data._create_dict(content)
feature_dictionary = data._create_feature_dictionary()